#### 4-4-1. 영상 - 영상 간의 연산
 - OpenCV에서도 사칙연산이 가능하다.
 - 한 픽셀의 범위값(0~255)을 초과하거나 미달하지 않도록 제한하기 위함.
 - dest = cv2.add(src1, src1[, dest, mask, dtype]): src1 + src2
    * src1: 입력 영상 1 또는 수
    * src2: 입력 영상 2 또는 수
    * dest: 출력영상
    * mask: 0이 아닌 픽셀만 연산
    * dtype: 출력 dtype
 - cv2.substract: src1 - src2
 - cv2.multiply: src1 * src2
 - cv2.divide: src1 / src2

In [ ]:
#practice. 영상의 사칙 연산
import cv2
import numpy as np

#연산에 사용할 배열 생성 -- ①
a = np.uint8([[200, 50]])
b = np.uint8([[100, 100]])

#Numpy 배열 직접 연산 -- ②    #255 -> 0 -> 1 -> 2-> ... -> 254 -> 255
add1 = a + b
sub1 = a - b
mult1 = a * 2
div1 = a / 3

#OpenCV API를 이용한 연산 -- ③
add2 = cv2.add(a, b)
sub2 = cv2.subtract(a, b)
mult2 = cv2.multiply(a, 2)
div2 = cv2.divide(a, 3)

#각 연산 결과 출력 -- ④
print(add1, add2)
print(sub1, sub2)
print(mult1, mult2)
print(div1, div2)

 - 다음 cv2.add 함수는 같은 내용임.
    
        c = cv2.add(a, b)
        c = cv2.add(a, b, None)
        cv2.add(a, b, c)

 - b += a와 같이 두 입력의 합산 결과를 입력 인자의 하나에 재할당하고 싶을 때는 아래와 같이 작성,
    
        cv2.add(a, b, b) or b = cv2.add(a, b)

In [ ]:
#practice. mask와 누적 할당 연산
import cv2
import numpy as np

#연산에 사용할 배열 생성
a = np.array([[1, 2]], dtype = np.uint8)
b = np.array([[10, 20]], dtype = np.uint8)
#두번째 요소가 0인 마스크 배열 생성
mask = np.array([[1, 0]], dtype = np.uint8)

#누적할당과의 비교연산
c1 = cv2.add(a, b, None, mask)   #mask의 두 번째 요소가 0이므로 2+20의 연산은 이루어지지 않고 0이다.
print(c1)
c2 = cv2.add(a, b, b, mask)      #b의 두번째 항목인 20을 그대로 갖는다.
print(c2)

#### 4-4-2. 알파 블렌딩

In [ ]:
#practice. 이미지 단순 합성
import cv2
import numpy as np
import matplotlib.pylab as plt

#연산에 사용할 이미지 읽기
img1 = cv2.imread('../img/wing_wall.jpg')
img2 = cv2.imread('../img/yate.jpg')

#이미지 덧셈
img3 = img1 + img2           #더하기 연산
img4 = cv2.add(img1, img2)   #OpenCV 함수

imgs = {'img1': img1, 'img2': img2,
       'img1 + img2': img3, 'cv2.add(img1, img2)': img4}

plt.figure(figsize = (10, 10))
#이미지 출력
for i, (k, v) in enumerate(imgs.items()):
    plt.subplot(2, 2, i+1)
    plt.imshow(v[:, :, ::-1])
    plt.title(k)
    plt.xticks([]); plt.yticks([])
    
plt.show()

 - ↑ img1 + img2 : 화소가 고르지 못하거나 색이 이상한 부분: 픽셀이 255를 초과한 영역
 - cv2.add(img1, img2)는 하얀 픽셀이 많아 좋은 결과로 볼 수 없음.

 - img1과 img2 두 영상을 합성하려면? 각각의 영상에 가중치 적용 필요!
 - alpha값 = 각 영상에 적용하는 가중치.
 - 가중치는 7:3, 6:4, 5:5 등과 같이 배분하는 방식. 
 
        g(x) = (1 - a)f0(x) + af1(x)
         - f0(x): 첫번째 이미지 픽셀값
         - f1(x): 두번쨰 이미지 픽셀값
         - a: 가중치(알파)
         - g(x): 합성 결과 픽셀값

- 위 함수를 OpenCV 함수로 구현

        cv2.addWeight(img1, alpha, img2, beta, gamma)
 * img1, img2: 합성할 두 영상
 * alpha: img1에 지정할 가중치(알파값)
 * beta: img2에 저장할 가중치, 흔히(1-alpha) 적용
 * gamma: 연산 결과에 가감할 상수. 흔히 0 적용.

In [ ]:
#practice. 50% 알파 블렌딩
import cv2
import numpy as np

alpha = 0.5    #합성에 사용할 알파값

#합성에 사용할 영상 읽기 -- ①
img1 = cv2.imread('../img/wing_wall.jpg')
img2 = cv2.imread('../img/yate.jpg')

#수식을 직접 연산해서 알파 블렌딩 적용 -- ②
blended = img1 * alpha + img2 * (1-alpha)
blended = blended.astype(np.uint8)   #소수점 발생을 제거하기 위함.
cv2.imshow('img * alpha + img2 * (1-alpha)', blended)

#addWeighted() 함수로 알파 블렌딩 적용 -- ③
dst = cv2.addWeighted(img1, alpha, img2, (1-alpha), 0)
cv2.imshow('cv2.addWeighted', dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#practice. 트랙바로 알파 블렌딩 
import cv2
import numpy as np

win_name = 'Alpha blending'    #창 이름
trackbar_name = 'fade'         #트랙바 이름

#트랙바 이벤트 핸들러 함수
def onChange(x):
    alpha = x/100
    dst = cv2.addWeighted(img1, 1-alpha, img2, alpha, 0)
    cv2.imshow(win_name, dst)
    
#합성 영상 읽기
img1 = cv2.imread('../img/man_face.jpg')
img2 = cv2.imread('../img/lion_face.jpg')

#이미지 표시 및 트랙바 붙이기
cv2.imshow(win_name, img1)
cv2.createTrackbar(trackbar_name, win_name, 0, 100, onChange)

cv2.waitKey()
cv2.destroyAllWindows()

#### 4-4-3. 비트와이즈 연산
 - bitwise(비트 단위) 연산: 두 영상을 합성할 때 특정 영역만 선택하거나 제외하는 등의 선별적인 연산.
   * bitwise_and(img1, img2, mask = None): 각 픽셀에 대해 비트와이즈 AND 연산
   * bitwise_or(img1, img2, mask = None): OR 연산
   * bitwise_xor(img1, img2, mask = None): XOR 연산
   * bitwise_not(img1, mask = None): Not 연산
     * img1, img2: 연산 대상 영상, 동일한 shape
     * mask: 0이 아닌 픽셀만 연산, 바이너리 이미지

In [ ]:
#practice. 비트와이즈 연산
import numpy as np
import cv2
import matplotlib.pyplot as plt

#연산에 사용할 이미지 생성
img1 = np.zeros( (200, 400), dtype = np.uint8)
img2 = np.zeros( (200, 400), dtype = np.uint8)
img1[:, :200] = 255        #오른쪽은 검은색(0), 왼쪽은 흰색(255)
img2[100:200, :] = 255     #위쪽은 검은색(0), 아래쪽은 흰색(255)

#비트와이즈 연산
bitAnd = cv2.bitwise_and(img1, img2)
bitOr = cv2.bitwise_or(img1, img2)
bitXor = cv2.bitwise_xor(img1, img2)
bitNot = cv2.bitwise_not(img1)

#Plot으로 결과 출력
imgs = {'img1':img1, 'img2':img2, 'and':bitAnd, 'or':bitOr, 'xor':bitXor, 'not':bitNot}
for i, (title, img) in enumerate(imgs.items()):
    plt.subplot(3, 2, i+1)
    plt.title(title)
    plt.imshow(img, 'gray')
    plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
#practice. bitwise_and 연산으로 마스킹하기
import numpy as np
import cv2
import matplotlib.pylab as plt

#이미지 읽기
img = cv2.imread('../img/girl.jpg')

#마스크 만들기
mask = np.zeros_like(img)                                   #원본 이미지와 동일한 shape, 0을 채워진 mask 배열
cv2.circle(mask, (150, 140), 100, (255, 255, 255), -1)      #하얀 원 생성
#         (대상 이미지, (원점 x값, 원점 y값), 반지름, (색상), 채우기)

#마스킹
masked = cv2.bitwise_and(img, mask)

#결과 출력
cv2.imshow('original', img)
cv2.imshow('mask', mask)
cv2.imshow('masked', masked)
cv2.waitKey()
cv2.destroyAllWindows()

#### 4-4-4. 차영상(image differencing)
 - 영상 빼기 영상: 두 영상의 차이 or 변화 확인 가능
 - 활용영역: 산업 현장에서 도면 차이 찾기, PCB(Printable Circuit Board) 회로의 오류 찾기, 카메라로 촬영한 영상에 실시간으로 움직임이 있는지 확인.
 - 차영상 결과는 절대값을 적용: 무조건 빼면 음수가 나올 수도 있음.

 - diff = cv2.absdiff(img1, img2)
   * img1, img2: 입력 영상
   * diff: 두 영상의 차의 절대값 반환

In [ ]:
#practice. 차영상으로 도면의 차이 찾아내기
import numpy as np
import cv2

#연산에 필요한 영상을 읽고 그레이 스케일로 변환
img1 = cv2.imread('../img/robot_arm1.jpg')
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('../img/robot_arm2.jpg')
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#두 영상의 절대값 차 연산
diff = cv2.absdiff(img1_gray, img2_gray)

#차 영상을 극대화하기 위해 스레시홀드 처리 및 컬러로 변환
_, diff = cv2.threshold(diff, 1, 255, cv2.THRESH_BINARY)  #1보다 큰 값은 모두 255로 바꾸고 색상을 컬러 스케일로 바꿈.
diff_red = cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)
#print(diff_red.shape)
diff_red[:, :, 2] = 0       #OpenCV는 RGB가 아니라 BGR이다. R값을 0으로 하니 빨간색임.
print(diff_red)
#두 번째 이미지에 변화 부분 표시
spot = cv2.bitwise_xor(img2, diff_red)   

#결과 영상 출력
cv2.imshow('img1', img1)
cv2.imshow('img2', img2)
cv2.imshow('diff', diff)
cv2.imshow('spot', spot)
cv2.waitKey()
cv2.destroyAllWindows()

#### 4-4-5. 이미지 합성과 마스킹

 - 합성의 과정
  * 1)전경이 될 영상과 배경이 될 영상에서 합성하고자 하는 영역만 떼어내는 작업: 마스크(mask) 필요.
  * 2)그것을 다시 합하는 작업

In [ ]:
#practice. 투명 배경 PNG 파일을 이용한 합성
#배경이 투명한 알파 채널 영상을 이용해서 영상 합성.
#4개 채널 중 마지막 채널에서 배경에 해당하는 영역은 0, 전경에 해당하는 영역은 255임.
#마스크를 이용해 전경과 배경을 오려낸다.

import cv2
import numpy as np

#합성에 사용할 영상 읽기, 전경 영상은 4채널 png 파일
img_fg = cv2.imread('../img/opencv_logo.png', cv2.IMREAD_UNCHANGED)
img_bg = cv2.imread("../img/girl.jpg")

#알파 채널을 이용해서 마스크와 역마스크 생성
_, mask = cv2.threshold(img_fg[:, :, 3], 1, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

#전경 영상 크기로 배경 영상에서 ROI 잘라내기
img_fg = cv2.cvtColor(img_fg, cv2.COLOR_BGRA2BGR)
h, w = img_fg.shape[:2]
roi = img_bg[10:10+h, 10:10+w]

#마스크 이용해서 오려내기
masked_fg = cv2.bitwise_and(img_fg, img_fg, mask = mask)
masked_bg = cv2.bitwise_and(roi, roi, mask = mask)

#이미지 합성
added = masked_fg + masked_bg
img_bg[10:10+h, 10:10+w] = added

cv2.imshow('mask', mask)
cv2.imshow('mask_inv', mask_inv)
cv2.imshow('masked_fg', masked_fg)
cv2.imshow('masked_bg', masked_bg)
cv2.imshow('added', added)
cv2.imshow('result', img_bg)
cv2.waitKey()
cv2.destroyAllWindows()

 - 모양에 따라 영역을 떼어낼 수도 있지만, 색상에 따라 영역을 떼어내야 하는 경우도 있다.
 - 이때는 컬러를 가지고 마스크를 만들어야 한다.
 - chroma keying: 색상을 이용한 마스크를 이용하는 것.
   * ex. 일기예보나 영화를 촬영할 때 우선 어떤 색상의 배경(=chroma key)을 두고 찍은 다음, 나중에는 원하는 배경과 합성하기.
 - HSV변환을 통해 원하는 색상 범위를 골라낼 수 있음.
 - dst = cv2.inRange(img, from, to): 범위에 속하지 않은 픽셀 판단
    * img: 입력 영상
    * from: 범위의 시작 배열
    * to: 범위의 끝 배열
    * dst: img가 from ~ to에 포함되면 255, 아니면 0을 픽셀값으로 하는 배열

In [ ]:
#practice. HSV 색상으로 마스킹
import cv2
import numpy as np
import matplotlib.pylab as plt

#큐브 영상을 읽어서 HSV로 변환
img = cv2.imread("../img/cube.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#색상별 영역 지정
blue1 = np.array([90, 50, 50])
blue2 = np.array([120, 255, 255])
green1 = np.array([45, 50, 50])
green2 = np.array([75, 255, 255])
red1 = np.array([0, 50, 50])       #HSV에서 빨강은 180을 기점으로 둘로 나뉘어 있으므로(0 ~ 15, 165 ~ 180)
red2 = np.array([15, 255, 255])    #색상 추출에도 두 번씩 사용됨.
red3 = np.array([165, 50, 50])
red4 = np.array([180, 255, 255])
yellow1 = np.array([20, 50, 50])
yellow2 = np.array([35, 255, 255])

#색상에 따른 마스크 생성
mask_blue = cv2.inRange(hsv, blue1, blue2)     #첫 번째 인자의 영상에서 두 번째, 세 번째 인자의 배열구간에 포함되면
mask_green = cv2.inRange(hsv, green1, green2)  #해당 픽셀의 값으로 255를 할당하고, 아니면 0을 할당함.
mask_red = cv2.inRange(hsv, red1, red2)        #그러므로 반환 결과는 바이너리 스케일이 되어 bitwise연산의 마스크로 사용하기에 적합함.
mask_red2 = cv2.inRange(hsv, red3, red4)
mask_yellow = cv2.inRange(hsv, yellow1, yellow2)

#색상별 마스크로 색상만 추출
res_blue = cv2.bitwise_and(img, img, mask = mask_blue)
res_green = cv2.bitwise_and(img, img, mask = mask_green)
res_red1 = cv2.bitwise_and(img, img, mask = mask_red)
res_red2 = cv2.bitwise_and(img, img, mask = mask_red2)
res_red = cv2.bitwise_or(res_red1, res_red2)
res_yellow = cv2.bitwise_and(img, img, mask = mask_yellow)

#결과 출력
imgs = {'original': img, 'blue': res_blue, 'green': res_green, 'red':res_red, 'yellow':res_yellow}
for i, (k, v) in enumerate(imgs.items()):
    plt.subplot(2, 3, i+1)
    plt.title(k)
    plt.imshow(v[:, :, ::-1])
    plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
#practice. 크로마 키 마스킹과 합성
import cv2
import numpy as np
import matplotlib.pylab as plt

#크로마 키 영상과 합성할 영상 읽기 -- ①
img1 = cv2.imread('../img/man_chromakey.jpg')
img2 = cv2.imread('../img/street.jpg')

#ROI 선택을 위한 좌표 계산(가운데에 위치하기 위한) -- ②
height1, width1 = img1.shape[:2]   #img.shape = (740, 756, 3)
height2, width2 = img2.shape[:2]
x = (width2 - width1) // 2
y = height2 - height1
w = x + width1
h = y + height1

#크로마 키 배경 영상에서 크로마 키가 있을 법한 영역을 10픽셀 정도로 지정. -- ③
chromakey = img1[:10, :10, :]   #남자 사진 왼쪽 끝 10x10 영역을 크로마 키 영역으로 어림잡음.
offset = 20

#크로마 키 영역와 영상 전체를 HSV로 변경 -- ④
hsv_chroma = cv2.cvtColor(chromakey, cv2.COLOR_BGR2HSV)
hsv_img = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)

#크로마 키 영역의 H값에서 offset만큼 여유를 두어서 범위 지정 
#offset값은 여러 차례 시도 후하여 경험적으로 얻어내야 함.
chroma_h = hsv_chroma[:, :, 0]
lower = np.array([chroma_h.min() - offset, 100, 100]) #18번 줄에서 어림잡아 선택한 영역의 색상값보다 더 넓은 색상을 선택할 수 있도록 offset만큼 가감.
upper = np.array([chroma_h.max() + offset, 255, 255]) #크로마 키의 색상값도 화면 전체적으로는 조금씩 다를 수 있기 때문이다.
                                                      #HSV 중 s, v값 선택 범위도 마찬가지이다.
#마스크 생성 및 마스킹 후 합성
mask = cv2.inRange(hsv_img, lower, upper)   #색상 값 중 최대값, 최소값을 범위로 지정하여 배경만 제거.
mask_inv = cv2.bitwise_not(mask)        
roi = img2[y:h, x:w]
fg = cv2.bitwise_and(img1, img1, mask = mask_inv)
bg = cv2.bitwise_and(roi, roi, mask = mask)
img2[y:h, x:w] = fg + bg

#결과 출력
cv2.imshow('chromakey', img1)
cv2.imshow('added', img2)
cv2.waitKey()
cv2.destroyAllWindows()

 - 이렇듯, 영상 합성에는 알파 블렌딩 or 마스킹이 필요한 경우가 대부분.
 - 그러나 적절한 알파값 선정, 마스킹 모양의 좌표나 색상값 선택에 많은 노력을 덜어줄 OpenCV의 함수가 있다.
 - dst = cv2.seamlessClone(src, dst, mask, coords, flags[, output])
   * src: 입력 영상(일반적으로 전경)
   * dst: 대상 영상(일반적으로 배경)
   * mask: 마스크, src에서 합성하고자 하는 영역은 255, 나머지는 0
   * coords: src가 놓여지기 원하는 dst의 좌표(중앙)
   * flags: 합성 방식
    * cv2.NORMAL_CLONE: 입력 원본 유지
    * cv2.MIXED_CLONE: 입력과 대상을 혼합
   * output: 합성 결과
   * dst: 합성 결과

In [ ]:
#practice. SeamlessClone으로 합성
import cv2
import numpy as np
#import matplotlib.pylab as plt

#합성 대상 영상 읽기 
img1 = cv2.imread('../img/drawing.jpg')   #합성시킬 이미지.
img2 = cv2.imread('../img/my_hand.jpg')   #img2 위에 img1이 합성됨.

#마스크 생성, 합성할 이미지 전체 영역을 255로 세팅
mask = np.full_like(img1, 255)     #img1 전체 영역을 255로 채움/. => 영역 전부가 합성의 대상임을 표현.
                                   #그러나, 가급적 합성하려는 영역 외에는 0으로 채우는 것이 더 좋다.
#합성 대상 좌표 계산(img2의 중앙)
height, width = img2.shape[:2]          
center = (width // 2, height // 2)

#seamlessClone으로 합성 -- ①          #img1을 img2에다가 mask에 지정된 영역만큼 center 좌표에 합성함.
normal = cv2.seamlessClone(img1, img2, mask, center, cv2.NORMAL_CLONE) #꽃 자체는 선명하나 주위가 흐림
mixed = cv2.seamlessClone(img1, img2, mask, center, cv2.MIXED_CLONE)   #주위도 선명하게 잘 합성됨.

#결과 출력      #알파값이나 마스크를 직접 정하지 않고도 잘 합성되어 편리함.
cv2.imshow('normal', normal)
cv2.imshow('mixed', mixed)
cv2.waitKey()
cv2.destroyAllWindows()